In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import os 

device = torch.device('mps')

## Resnet- 18 층 구현 
1. 블록구현
2. 데이터 불러오기
3. 모델 학습 

### resnet 층 1개 구조
![resnet](/Users/mac/AIworkspace/torchspace/resnet/image/resnet_st.png)
- each convolution + batch normalization + activation 


In [2]:

class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()

        ## 3x3 필터를 사용
        # in_planes : 입력 사진의 필터의 수 
        # planes : 출력 사진의 필터의 수 
        self.conv1 =  nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1 , bias=False )
        self.bin1 = nn.BatchNorm2d(planes)

        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bin2 = nn.BatchNorm2d(planes)

        # 입력값 그대로 출력하는 경우
        self.shortcut = nn.Sequential() # 항등함수인 경우 : 입력값 그대로 출력()
        

        # 항등함수가 아닌경우 : 차원을 맞춰주기 위해 1x1 conv 사용 , stride = 2인경우
        # stride=1이 아니면, 사이즈가 반으로 줄어들어야 하기 때문에 stride=1로 설정하여 conv2d 연산을 수행한다. 
        if stride != 1 or in_planes != planes :
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1 , stride=stride , bias=False),
                nn.BatchNorm2d(planes)
            )
    # forward : 잔차 + skip connection
    def forward(self, x):
        out = F.relu(self.bin1(self.conv1(x))) # 첫번째 conv -> batch -> relu
        out = self.bin2(self.conv2(out)) 
        out += self.shortcut(x) # 첫 입력값을 더함 = skip connection
        out = F.relu(out)
        return out    

## bottleneck
- 1x1 필터 : 채널을 줄이거나 키울 때 사용 , 채널의 방향의 선형변환만 수행된다. 
- 3x3 필터 : 피처맵의 특성을 추출한다. 

-> 1x1을 활용함으로써 연산량을 줄인다. 

-> 마지막 1x1에서 채널을 4배로 늘리는 이유? 
1. 더 풍부한 표현 학습 목적


In [3]:
# Bottleneck Block 정의
class Bottleneck(nn.Module):
    expansion = 4  # 마지막 1x1 Conv에서 4배 확장

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()

        # 1x1 Conv (Reduce)
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)

        # 3x3 Conv (Convolve)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        # 1x1 Conv (Expand)
        self.conv3 = nn.Conv2d(planes, planes * self.expansion, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * self.expansion)

        # Skip Connection (입출력 차원이 다르면 1x1 Conv 적용)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes * self.expansion:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes * self.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * self.expansion)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))  # 1x1 Conv
        out = F.relu(self.bn2(self.conv2(out)))  # 3x3 Conv
        out = self.bn3(self.conv3(out))  # 1x1 Conv
        out += self.shortcut(x)  # Skip Connection
        out = F.relu(out)
        return out


### 모델 구조
![resnet_structure](/Users/mac/AIworkspace/torchspace/resnet/image/resnet.png)

In [4]:
# ResNet-18 정의 
class Resnet18(nn.Module):
    def __init__(self, block, num_blcoks , num_classes =1000):
        super(Resnet18, self).__init__()
        self.in_planes = 64

        # 64개의 7x7 필터를 사용
        self.c1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.b1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Residual Blocks, make_layer 없이 구현 
        self.layer1 =  nn.Sequential(
            block(64, 64),
            block(64, 64)
        )
        self.layer2 =  nn.Sequential(
            block(64, 128, stride=2),
            block(128, 128)
        )
        self.layer3 =  nn.Sequential(
            block(128, 256, stride=2),
            block(256, 256)
        )
        self.layer4 =  nn.Sequential(
            block(256, 512, stride=2),
            block(512, 512)
        )
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        out = self.relu(self.b1(self.c1(x)))
        out = self.maxpool(out)

        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)

        out = self.avgpool(out)
        out = out.view(out.size(0), -1) # flatten
        out = self.linear(out)
        return out
    

Resnet18(BasicBlock, [2, 2, 2, 2]).to(device)

Resnet18(
  (c1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (b1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bin1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bin2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bin1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3)

In [5]:
# ResNet-50 네트워크 정의
class Resnet50(nn.Module):
    def __init__(self, block, num_blocks, num_classes=1000):  # ImageNet 기준 1000개 클래스
        super(Resnet50, self).__init__()
        self.in_planes = 64

        # 첫 번째 Conv (논문 그대로 7x7, stride=2 사용)
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)  # 3×3 Max Pooling 추가

        # ResNet 블록들 (논문 구조대로 설정)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)  # 56×56 유지
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2) # 56×56 → 28×28
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2) # 28×28 → 14×14
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2) # 14×14 → 7×7

        # Global Average Pooling (GAP) + FC Layer
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))  # 최종 1x1 Feature Map
        self.fc = nn.Linear(512 * block.expansion, num_classes)  # Fully Connected Layer

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)  # 첫 블록만 stride 적용
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion  # 현재 블록의 출력 채널을 다음 블록의 입력 채널로 설정
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))  # 7×7 Conv → ReLU
        x = self.maxpool(x)  # 3×3 Max Pooling

        x = self.layer1(x)  # 56×56 
        x = self.layer2(x)  # 28×28
        x = self.layer3(x)  # 14×14
        x = self.layer4(x)  # 7×7

        x = self.avgpool(x)  # 1×1
        x = torch.flatten(x, 1)  # Flatten for FC
        x = self.fc(x)  # Fully Connected Layer
        return x

# ResNet-50 생성 함수
def ResNet50():
    return Resnet50(Bottleneck, [3, 4, 6, 3])  # Bottleneck 구조 사용


## 데이터셋 불러오기

In [6]:
import torchvision
import torchvision.transforms as transforms

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)
     


/opt/anaconda3/envs/torch_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Files already downloaded and verified
Files already downloaded and verified


## 학습 

In [7]:
net = Resnet18()
net = net.to(device) 
net = torch.nn.DataParallel(net)
cudnn.benchmark= True

learning_rate= 0.1
file_name= 'resnet18_cifar10.pt'

criterion = nn.CrossEntropyLoss() # 손실함수 
optimizer = optim.SGD(net.parameters(), lr=learning_rate , momentum=0.9, weight_decay=0.0002)

def train(epoch):
    print('\n[ Train epoch: %d ]' % epoch) # 몇번째 에포크인지 
    net.train() # train 시작
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader): # input(이미지 데이터) + 정답 레이블로 구성되어있는 두 개의 텐서
        inputs, targets = inputs.to(device), targets.to(device) # gpu 설정 
        optimizer.zero_grad() # 이전 단계의 기울기 초기화 

        # forward propagation 
        benign_outputs = net(inputs) # 모델에 데이터를 전달하여 예측값 생성 
        loss = criterion(benign_outputs, targets) # 손실함수 계산 
        loss.backward() # 역전파 

        optimizer.step() # 가중치 업데이트 
        train_loss += loss.item() # 누적 손실 계산 
        _, predicted = benign_outputs.max(1) # 예측 결과 선택 -> 가장 높은 확률값 기준 

        total += targets.size(0) # 정답 개수 
        correct += predicted.eq(targets).sum().item() # 총 정답 개수 업데이트 
        
        # 100번째 batch 마다 중간 결과 출력  
        if batch_idx % 100 == 0:   
            print('\nCurrent batch:', str(batch_idx))
            # train 정확도 출력 : 정확하게 맞춘 샘플의 개수 합산 / 전체 배치 크기 
            print('Current benign train accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
            print('Current benign train loss:', loss.item())

    # 최종 결과 출력 
    print('\nTotal benign train accuarcy:', 100. * correct / total)
    print('Total benign train loss:', train_loss)


TypeError: Resnet18.__init__() missing 2 required positional arguments: 'block' and 'num_blcoks'

In [29]:
def test(epoch):
    print('\n[ Test epoch: %d ]' % epoch)
    net.eval()
    loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(test_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        # 전체 배치 크기 
        total += targets.size(0)

        # 결과 및 loss 
        outputs = net(inputs)
        loss += criterion(outputs, targets).item()

        # 가장 높은 확률을 갖는 것을 예측으로 
        _, predicted = outputs.max(1)
        # 정확하게 맞춘 샘플의 개수 합산 
        correct += predicted.eq(targets).sum().item()

    print('\nTest accuarcy:', 100. * correct / total)
    print('Test average loss:', loss / total)

    state = {
        'net': net.state_dict()
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/' + file_name)
    print('Model Saved!')


In [30]:
# 에포크가 올라갈수록 학습률을 더 낮춘다. 세밀하게 조정 
def adjust_learning_rate(optimizer, epoch):
    lr = learning_rate
    if epoch >= 100:
        lr /= 10
    if epoch >= 150:
        lr /= 10
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [22]:
# for epoch in range(0, 200):
for epoch in range(0, 20):
    adjust_learning_rate(optimizer, epoch)
    train(epoch)
    test(epoch)


[ Train epoch: 0 ]

Current batch: 0
Current benign train accuracy: 0.046875
Current benign train loss: 2.4866790771484375

Current batch: 100
Current benign train accuracy: 0.296875
Current benign train loss: 2.0104289054870605

Current batch: 200
Current benign train accuracy: 0.3125
Current benign train loss: 1.7229745388031006

Current batch: 300
Current benign train accuracy: 0.4375
Current benign train loss: 1.6816315650939941

Total benign train accuarcy: 31.224
Total benign train loss: 737.9201501607895

[ Test epoch: 0 ]

Test accuarcy: 37.42
Test average loss: 0.017374002707004548
Model Saved!

[ Train epoch: 1 ]

Current batch: 0
Current benign train accuracy: 0.5
Current benign train loss: 1.4128056764602661

Current batch: 100
Current benign train accuracy: 0.484375
Current benign train loss: 1.4949407577514648

Current batch: 200
Current benign train accuracy: 0.46875
Current benign train loss: 1.4096026420593262

Current batch: 300
Current benign train accuracy: 0.60156